## 01) 크롤러_시크먼트 카페

In [1]:
import pandas as pd
import pickle #pickle: 저장된 list그자체로 저장해서 list를 그 자체로 가져오기위하여
from tqdm import tqdm
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys #selenium keys 객체가 control키를 입력하게 해줌
import time
import pyperclip #복사, 붙여넣기 모듈로, 네이버 자동입력 방지문자 봇감지로직에 걸리지 않고 로그인

import warnings # 경고 알림 제거
warnings.filterwarnings("ignore", category=Warning) # 경고 알림이 뜨면 모두 무시합니다.

In [2]:
def find_last_page(driver, start_date, end_date, keyword):
    page_url = f"https://cafe.naver.com/parisienlook/ArticleSearchList.nhn?search.clubid=23451561&search.media=0&search.searchdate={start_date}{end_date}&search.defaultValue=1&userDisplay=50&search.option=0&search.sortBy=date&search.searchBy=0&search.query={keyword}&search.viewtype=title"
    driver.get(page_url)
    time.sleep(1)
    driver.switch_to.frame('cafe_main') # driver 게시판 영역으로 switch (iframe 태그:화면 속에 화면을 넣는 태그)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    next_btn = soup.select('div.prev-next a.pgR')
    page_start = 0
    while next_btn:
        page_start += 1
        page_num = 10 * page_start + 1
        page_url = f"https://cafe.naver.com/parisienlook/ArticleSearchList.nhn?search.clubid=23451561&search.media=0&search.searchdate={start_date}{end_date}&search.defaultValue=1&userDisplay=50&search.option=0&search.sortBy=date&search.searchBy=0&search.query={keyword}&search.viewtype=title&search.page={page_num}"
        driver.get(page_url)
        time.sleep(1)
        driver.switch_to.frame('cafe_main')
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        next_btn = soup.select('div.prev-next a.pgR')
    
    last_page = soup.select('div.prev-next a')[-1].text
    return int(last_page)

In [3]:
# keyword 입력 in 전체게시판에서 url&게시판 개괄적 내용 추출 함수 (List로저장)
def collect_row(driver, start_date, end_date, page_num, keyword): #start_dateend_date: 2021-05-072021-05-19
    
    page_url = f"https://cafe.naver.com/parisienlook/ArticleSearchList.nhn?search.clubid=23451561&search.media=0&search.searchdate={start_date}{end_date}&search.defaultValue=1&userDisplay=50&search.option=0&search.sortBy=date&search.searchBy=0&search.query={keyword}&search.viewtype=title&search.page={page_num}"
    driver.get(page_url)
    time.sleep(1)
    driver.switch_to.frame('cafe_main') # driver 게시판 영역으로 switch (iframe 태그:화면 속에 화면을 넣는 태그)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    body = soup.find_all('div', class_='article-board')[-1].tbody
    
    article_href = [div.a['href'] for div in body.find_all('div', class_='inner_list')] # articleid로 게시글링크연결될수있는데 안바꾸고 예전것 사용
    article_links = ['https://cafe.naver.com/parisienlook/' + res for res in article_href]
    category_id = [div.text for div in body.find_all('div', class_='inner_number')] #articleid로 게시글링크연결되는 키값이나 >> 게시글 조회용으로 수집함

    article_titles = [div.a.text.strip() for div in body.find_all('div', class_='inner_list')]
    nick_names = [td.text.strip() for td in body.find_all('td', class_='p-nick')]
    dates = [td.text for td in body.find_all('td', class_='td_date')]
    num_views = [td.text for td in body.find_all('td', class_='td_view')]
    
    df_ = {
        'article_links' : article_links,
        'category_id' : category_id,
        'article_titles' : article_titles,
        'nick_names' : nick_names,
        'dates' : dates,
        'num_views' : num_views,        
    }

    return df_

In [4]:
# v2. 게시판 구성이 달라서(게시판 타입 명시안되어있는 게시글) 아래 항목 추출되지 않는 비율 124개 중 1개로 낮으나 수집해보기로하여 함수 업데이트
# 단, href로 들어가는것과 article_id로 들어가는것 안의 태그 달라서, href작성한것으로 그대로 유지
# 댓글이 100개 넘는 경우에 댓글 페이지 2페이지로 넘어가나, 명품 댓글에 100개가 많이 없어서 pass

def collect_text(driver, article):
    
    driver.get(article)
    time.sleep(1)
    driver.switch_to.frame('cafe_main')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    try:
        try:
            article_type = soup.select_one('div.tit-box a.m-tcol-c').text.strip()  #게시판 종류
        except:
            article_type = soup.select_one('div.title_area').text.strip() #회원등급이 낮아 접근 불가 페이지(중고 프리미엄) 게시판
    except:
        article_type = 'N/A'
    
    try:
        try:
            num_comments = soup.select_one('div.reply-box a.reply_btn').text.strip().split()[1] #댓글수
        except:
            num_comments = soup.select_one('div.reply-box span.reply').text.strip().split()[1] #댓글수 v2(게시판 구성 다른것)
    except:
        num_comments = '0'
    
    try:
        article_content = soup.select_one('div.tbody.m-tcol-c').text.strip() #게시판 본문
    except:
        article_content = None
    
    try:
        comment_names = [commname.text.strip() for commname in soup.select('div.box-reply2 td.p-nick') if commname.text.strip() != ''] #댓글 작성자(삭제된 댓글 제외)
    except:
        comment_names = None
    
    try:
        comment_text = [commtext.text.strip() for commtext in soup.select('div.box-reply2 span.comm_body') if commtext.text.strip() != '' and commtext.text.strip()!= '삭제된 댓글입니다.'] #게시글당 댓글내용 이모티콘 있는부분과 삭제된 댓글 제외  
    except:
        comment_text = None
    
    comment_list = list(zip(comment_names, comment_text)) #댓글 json타입으로 저장 많이한다고함 {댓글id1: 댓글내용1,댓글id2: 댓글내용2}
    
    data = {
    'article_type' : article_type,
    'article_content' : article_content,
    'num_comment' : num_comments,
    'comment_list' : comment_list
        }

    return data

In [5]:
driver = webdriver.Chrome('../../../chromedriver.exe')

driver.get('https://www.naver.com/') #웹페이지 접속

# 로그인 버튼을 찾고 클릭합니다
login_btn = driver.find_element_by_class_name('link_login')
login_btn.click()
time.sleep(1)

# id, pw 입력할 곳을 찾습니다.
tag_id = driver.find_element_by_id('id')
tag_pw = driver.find_element_by_id('pw')
tag_id.clear()

# id 입력
tag_id.click()
pyperclip.copy('XXX') #id 복사
tag_id.send_keys(Keys.CONTROL, 'v') #id 붙여넣기
time.sleep(1)

# pw 입력
tag_pw.click()
pyperclip.copy('YYY')
tag_pw.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

# 로그인 버튼을 클릭합니다
login_btn = driver.find_element_by_id('log.login')
login_btn.click()

In [6]:
%%time

# 두 함수가 파라미터 공통이어 변수로 저장
start_date = '2021-08-01'
end_date = '2021-08-31'

# keyword = '%BA%D2%B0%A1%B8%AE' #불가리
# keyword = '%B1%EE%B8%A3%B6%EC%BF%A1' #까르띠에
# keyword = '%C6%BC%C6%C4%B4%CF' #티파니
keyword = '%B9%DD%C5%AC' #반클리프 키워드

last_page = find_last_page(driver, start_date = start_date, end_date= end_date, keyword = keyword)
print(last_page) #게시글 수가 50-60페이지(1페이지당 50개) 넘어가면 짤리므로 기간 좁혀서 수집

7
Wall time: 3.49 s


In [102]:
%%time
links = []
df_category_id = []
df_article_titles = []
df_nick_names = []
df_dates = []
df_num_views = []
df_article_type = []
df_article_content = []
df_num_comment = []
df_comment_list = []

for i in tqdm(range(1, last_page+1)):
    page = collect_row(driver, start_date = start_date, end_date= end_date, page_num=i, keyword = keyword) 
    for index, link in enumerate(page['article_links']):
        df_category_id.append(page['category_id'][index])
        df_article_titles.append(page['article_titles'][index])
        df_nick_names.append(page['nick_names'][index])
        df_dates.append(page['dates'][index])
        df_num_views.append(page['num_views'][index])
        
        article = collect_text(driver, link)
        df_article_type.append(article['article_type'])
        df_article_content.append(article['article_content'])
        df_num_comment.append(article['num_comment'])
        df_comment_list.append(article['comment_list'])

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [11:19<00:00, 97.12s/it]

Wall time: 11min 19s


In [103]:
data = pd.DataFrame(list(zip(df_category_id, df_article_type, df_article_titles, df_nick_names, df_dates, df_num_views, df_article_content, df_num_comment, df_comment_list)),\
                 columns = ['category_id', 'article_type', 'article_titles', 'nick_names', 'dates', 'num_views', 'article_content', 'num_comment', 'comment_list'])

In [104]:
print(len(data))

324


In [105]:
data.head()

,category_id,article_type,article_titles,nick_names,dates,num_views,article_content,num_comment,comment_list
0,2931364,명품 시계 토크,이 반클 시계 어떤가용?,리즈벨,2021.08.31.,"1,685",누르면 다리에서 남녀가 만나서 키스 하는건데용가격이 사악하지만 제눈엔 이뻐보이눈데....,29,"[(낙타악어새, 개취라고 생각해요~~제 취향은 ㄴ ㄴ ..ㅠ), (리즈벨작성자, 가..."
1,2930638,자유로운 토크,목걸이 입문 도와주세용!!,angelangel,2021.08.31.,890,명품 목걸이는 처음이에요30대 중반이고 예산500만원 아래에서 보고있어요.​제 스타...,6,"[(블랙금장, 부쉐론이 제일 괜찮아요 ~ 여러가지 하실거 아님 요거 하나로 ~ 캐주..."
2,2930591,"[중고][일반회원]팝니다,교환",[팝니다]\n \n ...,Tccic,2021.08.31.,436,판매가 완료된 상품입니다.\n\n\n\n\n\n[새제품] 18K 다이아몬드 가드링\...,2,"[(슈슈슈크림빵, 저요!!), (Tccic작성자, 네 챗드릴게요)]"
3,2930575,명품 골라주세요,목걸이 추천 부탁드려요,나룡,2021.08.31.,223,생일기념 목걸이 사려고해요(100~200사이)30대 후반에무난하고 차분한 스타일 추...,3,"[(꼼데빠 커플, 30대 시면 티파니 이쁠것 같아요!), (나룡작성자, 곧 40대라..."
4,2930428,보석 쥬얼리 토크,반클리프 색상고민,도리도리d,2021.08.31.,725,안녕하세요 반클 빼를리 뱅글 구매하려고 하는데 고민이 있어요..반클 로즈골드냐 옐로...,5,"[(우앵잉, 전 갠적으로 로골요..!), (치즈라면, 로골이요~), (해피한인생, ..."


In [106]:
data = data.drop_duplicates(subset=['article_titles', 'nick_names', 'dates'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 324 entries, 0 to 323
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   category_id      324 non-null    object
 1   article_type     324 non-null    object
 2   article_titles   324 non-null    object
 3   nick_names       324 non-null    object
 4   dates            324 non-null    object
 5   num_views        324 non-null    object
 6   article_content  324 non-null    object
 7   num_comment      324 non-null    object
 8   comment_list     324 non-null    object
dtypes: object(9)
memory usage: 25.3+ KB


In [107]:
save_excel_name = 'data/VCA_2021_08.xlsx'
data.to_excel(save_excel_name, index=False) #읽기용으로 엑셀 저장하고, 전처리 등 형태소분석 할때는 피클로 저장하고

In [108]:
save_file_name = "data/VCA_2021_08.pk"
with open(save_file_name, "wb") as f: #wb: write binary
    pickle.dump(data, f)